In [1]:
from datetime import timedelta
import pandas as pd
import json
import requests
import numpy as np
import datetime as dt

import math
import time

from ortools.constraint_solver import routing_enums_pb2
from ortools.constraint_solver import pywrapcp

import haversine


%reload_ext autoreload
%autoreload 2

import sys
sys.path.append('..')
from src.distance import get_haversine_distance, get_distance_aws
from src.co2 import co2_truck, co2_train

In [3]:
def create_dict_points(df):
    df_latlong = df[["Receiver name", 'Receiver longitude', 'Receiver latitude', "Sender weight (kg)"]]
    df_latlong.loc[-1] = df[["Shipper name",'Shipper longitude', 'Shipper latitude']].iloc[1].to_list() + [0]  # adding a row
    df_latlong.index = df_latlong.index + 1  # shifting index
    df_latlong.sort_index(inplace=True)
    df_latlong = df_latlong.set_index("Receiver name")
    dict_points = df_latlong.T.to_dict("list")
    return dict_points

In [5]:
def create_distance_matrix(dict_points):
    # create points format
    list_points = list(dict_points.items())
    if len(list_points)>100:
        distance_matrix = haversine.haversine_vector(list(dict_points.values()), list(dict_points.values()), comb=True)*1000
        return distance_matrix.astype(int)
    else: 
        # Create points format
        points = str(list_points[0][1][0])+','+str(list_points[0][1][1])
        for i in range(1, len(list_points)):
            points = points+';'+str(list_points[i][1][0])+','+str(list_points[i][1][1])

        # get distance matrix with osrm
        url = f'http://router.project-osrm.org/table/v1/driving/{points}?annotations=distance'
        r = requests.get(url)
        json.loads(r.content)
        res = r.json()
        return np.array(res['distances'], dtype=int)

In [4]:
def create_data_model(dict_points, num_vehicles):
    """Stores the data for the problem."""
    data = {}
    data['distance_matrix'] = create_distance_matrix(dict_points)
    data['speed']=83 #km/h
    data['time_matrix'] = data['distance_matrix'] / data['speed'] * 60
    data['demands'] = [int(item[2]) for item in list(dict_points.values())]
    capacity = 17000
    data['num_vehicles'] = num_vehicles
    data['vehicle_capacities'] = np.full(shape=data['num_vehicles'],fill_value=capacity,dtype=int)
    data['depot'] = 0
    data['customers'] = list(dict_points.keys())
    
    return data

In [ ]:
# delivery_checkpoints = list of (index, min_delivered_quantity)
def solve(data, delivery_checkpoints=[], ignored_time_constraint_nodes=[], num_vehicles=1, allowed_waiting_time_at_delivery=0, force_tour_debut_immediately=True, vehicle_capacity=[9999]):
    """Solve the CVRP problem."""

    # Create the routing index manager.
    manager = pywrapcp.RoutingIndexManager(len(data['distance_matrix']),
                                           num_vehicles, data['depot'])

    # Create Routing Model.
    routing = pywrapcp.RoutingModel(manager)
    
    model_parameters = pywrapcp.DefaultRoutingModelParameters()
    model_parameters.max_callback_cache_size = 2 * len(data["demands"]) * len(data["demands"])
    
    # Create Routing Model.
    routing = pywrapcp.RoutingModel(manager, model_parameters)

    def get_solution_details(data, manager, routing, solution):
      vehicles=[]
      time_dimension = routing.GetDimensionOrDie('Time')
      for vehicle_id in range(num_vehicles):
        index = routing.Start(vehicle_id)
        start_weight=data['total_demand']
        fuel_consumption = 0
        delivered_weight = 0
        node_index = manager.IndexToNode(index)
        nodes=[]
        node_times=[]
        while not routing.IsEnd(index):
            nodes.append(node_index)
            time_var=time_dimension.CumulVar(index)
            node_times.append((solution.Min(time_var),solution.Max(time_var)))
            delivered_weight += data['demands'][node_index]
            previous_node_index=node_index
            previous_index = index
            index = solution.Value(routing.NextVar(index))
            node_index = manager.IndexToNode(index)
            step_fuel_consumption = FuelConsumption(
                0,
                data['distance_matrix_meter'][previous_node_index,node_index]*1000,
                data['slope_matrix_gradient'][previous_node_index,node_index],
                CurbWeight+start_weight-delivered_weight,
                data['speed'])
            fuel_consumption+=step_fuel_consumption
        vehicles.append({
            "vehicle_id":vehicle_id,
            "CO2_emission":fuel_consumption*FuelToCO2,
            "fuel_consumption":fuel_consumption,
            "nodes":nodes,
            "node_times":node_times
        })
      return vehicles

    # Create and register a transit callback.
    def distance_callback(from_index, to_index):
        """Returns the distance between the two nodes."""
        # Convert from routing variable Index to distance matrix NodeIndex.
        from_node = manager.IndexToNode(from_index)
        to_node = manager.IndexToNode(to_index)
        return data['distance_matrix'][from_node][to_node]

    transit_callback_index = routing.RegisterTransitCallback(distance_callback)
    
    # Define cost of each arc.
    routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)

    # Add Capacity constraint.
    def demand_callback(from_index):
        """Returns the demand of the node."""
        # Convert from routing variable Index to demands NodeIndex.
        from_node = manager.IndexToNode(from_index)
        return data['demands'][from_node]
    demand_callback_index = routing.RegisterUnaryTransitCallback(demand_callback)
    routing.AddDimensionWithVehicleCapacity(
        demand_callback_index,
        0,  # null capacity slack
        data['vehicle_capacities'],  # vehicle maximum capacities
        True,  # start cumul to zero
        'Capacity')
    
    # the truck is obliged to have unloaded a certain quantity at different control points
    for delivery_checkpoint in delivery_checkpoints:
      routing.GetMutableDimension('Capacity').SetCumulVarSoftLowerBound(delivery_checkpoint[0], delivery_checkpoint[1], 100)

    # Create and register a transit callback.
    def time_callback(from_index, to_index):
        """Returns the travel time between the two nodes."""
        # Convert from routing variable Index to time matrix NodeIndex.
        from_node = manager.IndexToNode(from_index)
        to_node = manager.IndexToNode(to_index)
        total_time= data['time_matrix'][from_node][to_node]+data['delivery_time'][to_node]
        return total_time
    time_callback_index = routing.RegisterTransitCallback(time_callback)

    # Add Time Windows constraint.
    time = 'Time'
    routing.AddDimension(
        time_callback_index,
        allowed_waiting_time_at_delivery,  # allow waiting time
        24*600,  # maximum time per vehicle
        force_tour_debut_immediately,  # force start cumul to zero.
        time)
    time_dimension = routing.GetDimensionOrDie(time)
    # Add time window constraints for each location except depot.
    for location_idx, time_window in enumerate(data['time_windows']):
        if location_idx == data['depot']:
            continue
        index = manager.NodeToIndex(location_idx)
        if (not location_idx in ignored_time_constraint_nodes):
          time_dimension.CumulVar(index).SetRange(time_window[0], time_window[1])


    # Add time window constraints for each vehicle start node.
    depot_idx = data['depot']
    for vehicle_id in range(num_vehicles):
        index = routing.Start(vehicle_id)
        time_dimension.CumulVar(index).SetRange(
            data['time_windows'][depot_idx][0],
            data['time_windows'][depot_idx][1])
        
    # Instantiate route start and end times to produce feasible times.
    for i in range(num_vehicles):
        routing.AddVariableMinimizedByFinalizer(
            time_dimension.CumulVar(routing.Start(i)))
        routing.AddVariableMinimizedByFinalizer(
            time_dimension.CumulVar(routing.End(i)))


    # Setting first solution heuristic.
    search_parameters = pywrapcp.DefaultRoutingSearchParameters()
    search_parameters.first_solution_strategy = (
        routing_enums_pb2.FirstSolutionStrategy.GLOBAL_CHEAPEST_ARC)
    search_parameters.time_limit.FromSeconds(100)

    # Solve the problem.
    solution = routing.SolveWithParameters(search_parameters)

    # Print solution on console.
    if solution:
      return get_solution_details(data, manager, routing, solution)
    else:
      return None

In [2]:
df = pd.read_csv('../data/processed/poc3.csv', parse_dates = ["Pickup date"])

df_clients = df[['Receiver longitude', 'Receiver latitude']].drop_duplicates().reset_index()
df_clients.index += 1
df_clients= df_clients.reset_index().drop(["index"], axis = 1).rename({"level_0":"Receiver name"}, axis = 1)
df_clients["Receiver name"] = "C"+df_clients["Receiver name"].astype(str)

df_dc = df[['Shipper longitude', 'Shipper latitude', "DC country"]].drop_duplicates().reset_index()
df_dc.index += 1
df_dc = df_dc.reset_index().drop(["index"], axis = 1).rename({"level_0":"Shipper name"}, axis = 1)
df_dc["Shipper name"] = "DC"+df_dc["Shipper name"].astype(str)

df = df[['Pickup date','Country', 'Sender weight (kg)','Shipper longitude','Shipper latitude', 'Receiver longitude', 'Receiver latitude']].merge(df_clients, on = ['Receiver longitude', 'Receiver latitude'], how = "left").merge(df_dc, on = ['Shipper longitude', 'Shipper latitude'], how = "left")

C:\Users\marle\AppData\Local\Temp\ipykernel_23580\3938569011.py:1: DtypeWarning: Columns (1,5,14) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('../data/processed/poc3.csv', parse_dates = ["Pickup date"])
